In [8]:
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd 
import os 

In [9]:
os.chdir('your directory')

뉴스기사 제목, 날짜, 링크 가져오기 

In [ ]:
def finding_html(soup): 

    titles = soup.findAll('h2', {'class' : 'categoryArticle__title'})
    title_text = [title.get_text() for title in titles] 
  

    dates = soup.findAll('p', {'class' : 'categoryArticle__meta'})
    date_text = [date.get_text() for date in dates]


    link_text = []       
    for x in soup.findAll('div', {'class' : 'categoryArticle'}):
        links = x.find('a')['href']
        link_text.append(links)

    add = {'date' : date_text, 'title' : title_text, 'link' : link_text}

    result = pd.DataFrame(add)
    
    return result 



In [ ]:
for page_number in range(1,251): 
    url = 'https://oilprice.com/Energy/Crude-Oil/Page-{}.html'.format(page_number)
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    
    df = finding_html(soup) 
    
    print('다운 받고 있습니다*********')
    df.to_csv('link/2020_' + str(page_number) + '.csv', sep=',', encoding='utf-8-sig')

뉴스 기사본문 가져오기

In [ ]:
def find_content(links): 
    
    content_list = []
    for i in range(0,len(links)): 
        url = links[i]
        html = requests.get(url).text
        soup = BeautifulSoup(html)
        contents = soup.find('div', {'id' : 'article-content'}).get_text()
        content_list.append(contents)
     
    return content_list   


for file in os.listdir():
    for file_num in range(1,251): 
        raw_data = pd.read_csv('link/2020_{}.csv'.format(file_num))        
        links = raw_data['link']
        news_content_list = find_content(links)
        result = pd.DataFrame({'content' : news_content_list})          
        print('다운 받고 있습니다*********')
        result.to_csv('content/2020_' + str(file_num) + '.csv', sep=',', encoding='utf-8-sig')

CSV파일로 저장하기 

In [ ]:
for i in range(1,251): 
    # data with date, link, url 
    left_data = pd.read_csv('link/2020_{}.csv'.format(i), names=['index', 'date', 'title', 'link'], dtype=str, header=0)
    left_data = left_data.set_index('index')

    # data with content 
    right_data = pd.read_csv('content/2020_{}.csv'.format(i), names=['index', 'content'], dtype=str, header=0)
    right_data = right_data.set_index('index')

    aggregated_df = pd.merge(left_data, right_data, how='outer', on='index')
    
    print('파일 생성 중*********')
    aggregated_df.to_csv('link_and_content/2020_{}.csv'.format(i), sep=',', encoding='utf-8-sig')

CSV 파일 합치기 

In [ ]:
import pandas as pd  
import glob  
import os 
import re

all_data = pd.DataFrame()  
for f in glob.glob('link_and_content/2020_*.csv'): # 예를들어 202001, 202002, 202003 로 된 파일이면 2019_*  
    df = pd.read_csv(f, dtype=str)  
    all_data = all_data.append(df, ignore_index=False) 
    all_data.to_csv('link_and_content/aggregated.csv', mode= 'w', encoding='utf-8-sig')
    
    
    
"""
# 기존 파일 삭제
os.chdir('./link_and_content')
for filename in os.listdir():
    result = re.search('[0-9]+.csv', filename)
    if result:
        os.remove(filename) 
"""    